In [9]:
import requests
import os
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import datetime, requests
import json
from getpass import getpass
import subprocess
import json

In [8]:
#consumo de la api local para bluekeep
#se consume el url de la api local, oero no la global local, para usar la global local en la api usada con python, 
#se le agrega el ejecutor de la varible inicializada con flask, donde esta el debug, se le agrega un host 0.0.0.0, para ejecutar
#la api en un entorno local pero dentro de la red, usando la ip pública
url = "http://127.0.0.1:5000/bluekeep_Dectect_Udec"
# se pude la ip y e nombre dle contenedor, ay que tener en cuanta que la base de datos en este momento ya debe estar creada
ipI = input("Digite la ip: ")
nameContainer = input("Digite el nombre del contenedor: ")
#se hace un pequeño objeto de datos que va a tener el ip ingresado y el puerto por defecto 3389
data ={
    "ip": ipI,
    "port":3389
}
# en este try se encapsula el valor devuelto por la api en una variable llamada response, que junto con la importacion requests se 
# llama el metodo post que es el encargado de determinar si la ip es vulnerable  o no
try:
    response = requests.post(url,json=data)
    dat = response.json()
# despues de convertir la respuesta en json, se trata la llave status, que trae una cadena de dos indices, y se llama al indice 1 que es el 
# encargado de traer la respuesta de vulnerabilidad, seguido a esto se ejecutan crea el contenedor, y el objeto que se enviará al metodo de 
# añadir items, el unico datos importante que no puede faltar es la respuesta en la variable cont, que tiene el contenedor ya creado,
# esete comando se ejeutara si y solo si, la ip es vulnerable a bluekeep
    if dat['status'][1] == 'VULNERABLE':
        cont = createNewContaniner(nameContainer)
        datAS = {
           'ip' : ipI,
           'port' : data['port'],
           'status' : dat['status'][1],
           'id': nameContainer,
           'name' : nameContainer,
           'account' : 'bluekeep',
           'container' : cont
        }
        addNewItem(datAS,cont,ipI)
# en el caso que la ip no sea vulnerable se imprime ese mensaje
    else:
        print('La el dispositivo de esta ip está apagado, y se encuentra parchado para esta vulnerabilidad')
        
   #56 print(dat['status'][1])
except requests.exceptions.HTTPError as http_err:
    print(f'Error HTTP: {http_err}')
    

#ejemplos de vulnerabilidades 125.87.87.22 - 220.90.17.83

Digite la ip:  75.58.232.10
Digite el nombre del contenedor:  bluekeep1


Container with id 'bluekeep1' created


Digite el nombre del item:  prueba1



Reading Item by Id



Cuál es la contraseña Máquina ;):  ········


[sudo] password for nicolasrodrigeztorres04: 

 Google Android 7.X (90%), Linux 3.X (90%)

item prueba1 fue creado con exito


In [10]:
#operaciones adicionales
# informacion de la ip con io info
#esta api fue obtenida de rapidApi, junto con la llave y el headers, se hace el llaamdo a la api dentro de un try catch, se opbtiene el vaor con 
# la variable resp, y se convierte a json con la variable response y la libreria json, luego se creal el objeto object co las llaves y la 
# informacion que consideré reelevante para guardar en mi base de datos, esta funcion retornará ese objeto si todo sale como se plantea
def ipInfo(ip):
    try:
        
        urlI = "https://community-neutrino-ip-info.p.rapidapi.com/ip-info"
        payload = {
    	"reverse-lookup": "checked",
    	"ip": ip
        }
        headers = {
    	"content-type": "application/x-www-form-urlencoded",
    	"X-RapidAPI-Key": "81eb58f566mshd228c280cb28728p1aa7c2jsnc1e57849a6ce",
    	"X-RapidAPI-Host": "community-neutrino-ip-info.p.rapidapi.com"
        }
        resp = requests.post(urlI, data=payload, headers=headers)
        response = resp.json()
        object = {
            'country': response['country'],
            'city' : response['city'],
            'region' : response['region'],
            'timeZone' : response['timezone'],
            'latitude' : response['latitude'],
            'longitude' : response['longitude']
        }
        return object
    except requests.exceptions.HTTPError as err:
        print(f'Error HTTP: {err}')

# uso de nmap
# Se trató de usar nMap for python  pero datos a que no tenia los permisos respectivos con jupyter no podia ejecutar comandos 
# relacionados con tcp/ip, la funcion pide el ip y un argumento que está relacionado con la accion que ejecutará nMap, las operaciones
# se ejecutan en un try catch para manejar los errores de consola, y que el proceso de la aplicacion no se vea afectado por algun error
def nMap(ip,argument):
    try: 
# aqui se pide la contraseña del super usuario con una libreria llamada get pass, es solo para que al momento de escribir no salgan letras
# sino que salgan puntos, como cuando se dijita la contraseña en cualquier login bien hecho 
        password = getpass("Cuál es la contraseña Máquina ;): ")
# este comando se divide en tres partes, y se ejecutara en la consola para llamar una accion de nmap
# en la parte del echo, es la encargada de estar en espera a que la consola pida introducir algo, en este caso se asigana la contraseña ya 
# pedida, la segunda parte es donde se ejecuta el comando que llama a nmap con permisos root, se le asigan un argumento, y con el 
# --osscan-limit se limita a la busqueda de solo el sistema operativo, la tercera parte, con la palabra reservada greep, se filtra 
# la respuesta a un running(just guessing ) la cual trae la llave relacionada con el sistema operativo
        comand = f'echo "{password}" | sudo -S nmap {argument} --osscan-limit {ip} | grep "Running (JUST GUESSING)" '
# de aqui para abajo se obtiene el output con la libreria subprocess y un check_output, se activa el shell para permitir las interaccion con
# consola, luego se convierte la respuesta a string, y se le hace un split en donde esta el :, y se retorna la aprte del split que corresponde
# al indice 1
        output = subprocess.check_output(comand, shell=True)
        imp = output.decode('utf-8')
        sO = imp.split(':')
        return sO[1]
# de no poder ejecutarse correctamente, simplemente se retorna el mensaje de No encontrado para guardar en la abse de datos 
    except subprocess.CalledProcessError as e:
        return 'No encontrado'


In [11]:
### creacion del contenedor para las pruebas y todo lo que tiene que ver con azure
#definicion de permisos y conexion
# aqui se gregan las credenciales y el nombre de la base de datos a crearse o a llamer
settings = {
    'host': os.environ.get('ACCOUNT_HOST', 'https://nicolas-db.documents.azure.com:443/'),
    'master_key': os.environ.get('ACCOUNT_KEY', 'aTCvY6TlGZpjnRAcvNDXDjqNFsFE6YMN159oIsiNx7ETUL0eeSnDQeu5WttvCuZAxtgHAD0TKptbACDboItCIQ=='),
    'database_id': os.environ.get('COSMOS_DATABASE', 'nicolasdb'),
}

#Asignación de valores a variables internas
HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
    
try:
    # aqui se creal el cliente que es como la puerta de entrada a la base de datos 
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    # con esa misma puerta de entrada se hace un llamado a la base de datos y se guarad en esa variable db
    db = client.get_database_client('nicolasdb')
except exceptions.CosmosHttpResponseError as e:
        print('\nexpecion controlada: '.format(e.message))

def createNewContaniner(name):
# en esta parte se crea el contenedor con la base de datos ya obtenida, pero si no se ejecuta el comando de crear, quiere decir
# que el contenedor ya está creado o paso algo mas, pero nuestro cath, lo que haces procesar que el contenedor ya está creado y de ser asi,
# simplemenet lo llama y lo retorna, y en cualquiera de los casos siempre se va a retornar el contenedos
        try:
            container = db.create_container(id=name, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(name))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(name)
            print('Container with id \'{0}\' was found'.format(name))
        finally:
            return container

# esta es la funcion encargada de crear el item, esta pide el contenedor, un objeto y la ip
def addNewItem(object,container,ip):
        try:
          #   print("Al añadir ",ip)
# primero se pone en ejecucion un while que va a correr infinitamente hasta que se cumpla la lavildacion que estpa entro de este,
# primero se pide el nombre del item luego se crea un objeto llamdo info, con el contenedor, el nombre y el account es la subdivision,
# luego de esto se crean dos variables que va a retornar una funcion, la cual es read_item, la cual se encarga de mirar si el item existe,
# esta retorna uno, si existe retorna true y el while vuelve a ejecutar la funcion desde el principio, de retornar false, esto queire decir, que 
# el item no existe, entonces se entra al if porque cumple con la validación, la cual ejecuta la funcion de nmap ya explicada, y se le 
# envia el atributo de la ip, y el -O que es el encargado de revisar los sitemas operativos en nmap, luego de esto se crea un objeto,
# que es ya el que se va a designar para crear el item en la base de datos, guaradando la data de el tiempo en que se ejecutó, se guarda el 
# sistema oprativo, que es repuesta de la funcion de nmap, y una llave ipinfo que llama a la funcion de ip info, esto con el fin de,
# separar todo lo que retorna nustra funcion en una sola llave 
             while True:
                 name = input('Digite el nombre del item: ')
                 inf = {
                     'container' : container,
                     'id' : name,
                     'account' : 'bluekeep'
                 }
                 bol,result = read_item(inf)
                 if bol == False:
                         nm = nMap(ip,'-O')
                         dataS = {'id' : name,
                        'partitionKey' : "bluekeep",
                        'date' : datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                        'port' : object['port'],
                        'systemOp' : nm,
                        'status' :object['status'],
                        'ipInfo' : ipInfo(object['ip'])
                                 }
                         print(nm)
                         container.create_item(body=dataS)
                         break
            
             
             print(f"item {name} fue creado con exito")
        except exceptions.CosmosResourceExistsError as err:
            print('Container with id \'{0}\' was found'.format(err))
    
#el objecto debe dar el contenedor <3
# esta funcion se enarga de eliminar items, solo con le nombre o id, el contenedor y el account
def deleteItem(object):
    try:
        object['container'].delete_item(item=object['id'], partition_key=object['account'])
        print('item eliminado')
    except exceptions.CosmosHttpResponseError as e:
        error = str(e)
        print("Excepción controlada: "+error)

#leer item
# esta es la funcion que lee la base de datos en busqueda de un item en especifico
def read_item(object):
# todo lo que se ejecuta dentro del try no es más que un llamado o busqueda de un item en específico, en casod e encontrarlo,
# se retorna un boleano verdadero y una respuesta que es lo que trae ese item, en caso de que se llegase a usar en con fines de saber que hay 
# en el item, en caso de no encontrarse se entra en la excepcion de notfound que trae cosmos, y se retorna falso seguido de un mensaje,
# de no existencia del item
    try:
        print('\nReading Item by Id\n')
        response = object['container'].read_item(item=object['id'], partition_key=object['account'])
        ##print('Partition : {0}'.format(response))
        return True,response
    except exceptions.CosmosResourceNotFoundError:
        return False,'El item no fue encontrado'

In [4]:
#cont = createNewContaniner('pruebaBluekeep1')
#obj = {
#    'container' : cont ,
#    'id' : 'pruebaitem',
#    'account' : 'bluekeep'
#}
#bol,response = read_item(obj)
#print(response)

 

In [ ]:
#operaciones de prueba, recordar simbre usar primero el crador de contenedores para recuperar el contenedor creado
#cont = createNewContaniner("prueba2MAR")
#object = {
#    'id': 'pruebaitem1',
#    'name' : 'pruebaitem1',
#    'account' : 'bluekeep',
#    'container' : cont
#}
#deleteItem(object)
#addNewItem("1",cont)
#read_item(object)

In [ ]:
#ipInfo('121.33.10.8')
#nameContainer = input("Digite el nombre del contenedor: ")
#dn = nMap('118.68.123.237','-O')
#print(dn)
